In [12]:
import geopandas as gpd
from geopandas import GeoDataFrame as GDF

In [13]:
ca_counties = gpd.read_file("examples/sjoin/data/ca-counties.geojson")
ca_counties.head()

,OBJECTID,COUNTY_NAME,COUNTY_ABBREV,COUNTY_NUM,COUNTY_CODE,COUNTY_FIPS,ISLAND,GlobalID,SHAPE_Length,SHAPE_Area,geometry
0,1,Alameda,ALA,1,01,001,None,{E6F92268-D2DD-4CFB-8B79-5B4B2F07C559},2.538264,0.217411,"MULTIPOLYGON (((-122.27125 37.90503, -122.2702..."
1,2,Alpine,ALP,2,02,003,None,{870479B2-480A-494B-8352-AD60578839C1},2.170420,0.198471,"MULTIPOLYGON (((-119.58667 38.71420, -119.5865..."
2,3,Amador,AMA,3,03,005,None,{4F45B3A6-BE10-461C-8945-6B2AAA7119F6},2.924268,0.161961,"MULTIPOLYGON (((-120.07246 38.70276, -120.0724..."
3,4,Butte,BUT,4,04,007,None,{44FBA680-AECC-4E04-A499-29D69AFFBD4A},4.210365,0.455899,"MULTIPOLYGON (((-121.07661 39.59729, -121.0794..."
4,5,Calaveras,CAL,5,05,009,None,{D11EF739-4A1E-414E-BFD1-E7DCD56CD61E},2.989673,0.275908,"MULTIPOLYGON (((-120.01792 38.43586, -120.0178..."


In [14]:
ca_powerplants = gpd.read_file("examples/sjoin/data/ca-powerplants.geojson")
ca_powerplants.head()

,OBJECTID_1,CECPlantID,PlantName,EIAPlantID,RPS_Eligible,SmallPlant,Peaker,Small_Hydro,SolarThermal,Pumped_Storage,...,Zip,Capacity_Latest,Units,PriEnergySource,LastReportedYear,Lon_WGS84,Lat_WGS84,Annual_Hyperlink,Monthly_Hyperlink,geometry
0,1,B0001,Vaca Dixon Battery Storage,59256,0.0,1.0,0.0,0.0,0.0,0.0,...,95688.0,2.4,Unit 1,BAT,2013.0,-121.921635,38.400098,https://repository.energy.ca.gov/powerplantsge...,https://caenergy.maps.arcgis.com/sharing/rest/...,POINT (-121.92163 38.40010)
1,2,B0002,Tehachapi Storage Project,59661,0.0,1.0,0.0,0.0,0.0,0.0,...,93561.0,NaN,None,None,NaN,-118.380000,35.123334,https://caenergy.maps.arcgis.com/sharing/rest/...,https://caenergy.maps.arcgis.com/sharing/rest/...,POINT (-118.38000 35.12333)
2,3,B0003,Yerba Buena Battery,59257,0.0,1.0,0.0,0.0,0.0,0.0,...,95121.0,NaN,None,None,NaN,-121.750114,37.307228,https://caenergy.maps.arcgis.com/sharing/rest/...,https://caenergy.maps.arcgis.com/sharing/rest/...,POINT (-121.75011 37.30723)
3,4,B0004,Millikan Avenue BESS,60760,0.0,1.0,0.0,0.0,0.0,0.0,...,92606.0,NaN,None,None,NaN,-117.832291,33.691010,https://caenergy.maps.arcgis.com/sharing/rest/...,https://caenergy.maps.arcgis.com/sharing/rest/...,POINT (-117.83229 33.69101)
4,5,B0005,Mira Loma BESS A,60661,0.0,0.0,0.0,0.0,0.0,0.0,...,91761.0,NaN,None,None,NaN,-117.559669,34.004651,https://caenergy.maps.arcgis.com/sharing/rest/...,https://caenergy.maps.arcgis.com/sharing/rest/...,POINT (-117.55967 34.00465)


In [15]:
sjoined = gpd.sjoin(ca_powerplants, ca_counties, how="left", predicate="within")
# sjoined.head()
isinstance(dict(sjoined.dtypes)['geometry'], gpd.array.GeometryDtype)

# sjoined.plot()

True

In [16]:
from typing import Iterable, Optional

def synth_first(synth: Iterable[Optional[object]]) -> Optional[object]:
    return next(filter(bool, synth), None)

def print_all_synth(synth: Iterable[Optional[object]]) -> object:
    print('Warning, print_all may take a loooong time')
    any(map(print, filter(bool, synth)))
    print('Done.')

In [17]:
def synth_slim(res: GDF, target: GDF) -> None | GDF:
    try:
        return res[target.columns]
    except KeyError:
        return None

In [18]:
def synth_sjoin(l: GDF, r: GDF, t: GDF):
    query_preds = l.sindex.valid_query_predicates & r.sindex.valid_query_predicates
    for h in ('left', 'right', 'inner'):
        for p in query_preds:
            res: GDF = gpd.sjoin(l, r, how=h, predicate=p)
            slimmed = synth_slim(res, t)
            if t.equals(slimmed):
                yield ('sjoin', h, p)
            else:
                yield None


print(synth_first(synth_sjoin(ca_powerplants, ca_counties, sjoined)))

('sjoin', 'left', 'within')


In [19]:
import pandas as pd

geo_states = gpd.read_file('examples/merge/data/tl_2021_us_state.shp')
state_shapes = geo_states[['geometry', 'GEOID', 'NAME']]

state_finances = pd.read_csv('examples/merge/data/state_gov_finances.csv')[['GEO_ID', 'AMOUNT', 'YEAR', 'GOVTYPE', 'AGG_DESC']]
state_finances = state_finances[
    (state_finances['YEAR'] == '2020')
    & (state_finances['GOVTYPE'] == '002')
    & (state_finances['AGG_DESC'] == 'SF0001')
]
state_finances.update(
    state_finances['GEO_ID'].map(lambda geo_id: geo_id[-2:])
)
states_with_revenue = state_finances.merge(state_shapes, right_on='GEOID', left_on='GEO_ID')
states_with_revenue.head()

,GEO_ID,AMOUNT,YEAR,GOVTYPE,AGG_DESC,geometry,GEOID,NAME
0,01,36908937,2020,002,SF0001,"POLYGON ((-85.48830 30.99706, -85.48917 30.997...",01,Alabama
1,02,10421195,2020,002,SF0001,"MULTIPOLYGON (((-179.23023 51.28393, -179.2256...",02,Alaska
2,04,50547601,2020,002,SF0001,"POLYGON ((-110.75069 37.00301, -110.74193 37.0...",04,Arizona
3,05,28572105,2020,002,SF0001,"POLYGON ((-90.95577 34.11871, -90.95451 34.117...",05,Arkansas
4,06,413372904,2020,002,SF0001,"MULTIPOLYGON (((-116.10617 32.61848, -116.1085...",06,California


In [20]:
from collections import defaultdict
from itertools import product
from typing import Generator

def cols_by_dtype(frame: pd.DataFrame) -> dict[type, set[str]]:
    out = defaultdict(set)
    for k, v in dict(frame.dtypes).items():
        out[v].add(k)
    return dict(out)

def col_mapping_gen(l: pd.DataFrame, r: pd.DataFrame) -> Generator[tuple[str, str], None, None]:
    'Returns a generator of pairs of potentially equal column names'
    l_types = cols_by_dtype(l)
    r_types = cols_by_dtype(r)
    for k, l_v in l_types.items():
        if r_v := r_types.get(k, None):
            yield from product(l_v, r_v)



In [21]:
def synth_merge(l: pd.DataFrame, r: pd.DataFrame, t: pd.DataFrame):
    'Trys to merge all columns with matching dtype'
    for l_col, r_col in col_mapping_gen(l, r):
        for h in ('left', 'right', 'inner', 'outer'): # 'cross' throws weird exception
            res = pd.merge(l, r, how=h, left_on=l_col, right_on=r_col)
            slimmed = synth_slim(res, t)
            if t.equals(slimmed):
                yield ('merge', h, l_col, r_col)
            else:
                yield None

print_all_synth(synth_merge(state_finances, state_shapes, states_with_revenue))

Warning, print_all may take a loooong time
('merge', 'left', 'GEO_ID', 'GEOID')
('merge', 'inner', 'GEO_ID', 'GEOID')
Done.


In [22]:
def synth_binop(l: pd.DataFrame, r: pd.DataFrame, t: pd.DataFrame):
    merge_gen = synth_merge(l, r, t)
    if not isinstance(l, GDF) or not isinstance(r, GDF):
        yield from merge_gen
    else:
        sjoin_gen = synth_sjoin(l, r, t)
        done = ()
        while True:
            m_next = next(merge_gen, done)
            if m_next is done:
                yield from sjoin_gen
                break
            else:
                yield m_next
            
            s_next = next(sjoin_gen, done)
            if s_next is done:
                yield from merge_gen
                break
            else:
                yield s_next


print_all_synth(synth_binop(state_finances, state_shapes, states_with_revenue))
print_all_synth(synth_binop(ca_powerplants, ca_counties, sjoined))

Warning, print_all may take a loooong time
('merge', 'left', 'GEO_ID', 'GEOID')
('merge', 'inner', 'GEO_ID', 'GEOID')
Done.
Warning, print_all may take a loooong time
('sjoin', 'left', 'within')
('sjoin', 'left', 'intersects')
